# Fast Inference

## Install Requirements


In [2]:
%pip install airllm==0.9.5

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: nb-black 1.0.7 has a non-standard dependency specifier black>='19.3'; python_version >= "3.6". pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of nb-black or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


## Generate Text


In [1]:
from airllm import AirLLMLlama2


def generate_text(
    prompt: str = "What is the capital of United States?",
    model_name: str = "garage-bAInd/Platypus2-70B-instruct",
    max_tokens: int = 2048,
    **kwargs
):
    # The model_name can be the model path or the hugging face model repo id.
    model = AirLLMLlama2(model_name)
    input_tokens = model.tokenizer(
        [prompt],
        return_tensors="pt",
        return_attention_mask=False,
        truncation=True,
        max_length=max_tokens,
        padding=True,
    )
    # Count input tokens
    input_tokens_count = input_tokens["input_ids"].shape[1]
    generation_output = model.generate(
        input_tokens["input_ids"].cuda(),
        max_new_tokens=max_tokens - int(input_tokens_count),
        use_cache=True,
        return_dict_in_generate=True,
        **kwargs
    )
    output = model.tokenizer.decode(generation_output.sequences[0])
    return output


text = generate_text(
    prompt="What is the capital of United States?",
    model_name="garage-bAInd/Platypus2-70B-instruct",
    max_tokens=2048,
)
print(text)

Fetching 25 files:   0%|          | 0/25 [00:00<?, ?it/s]

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:33                                                                                   │
│                                                                                                  │
│   30 │   return output                                                                           │
│   31                                                                                             │
│   32                                                                                             │
│ ❱ 33 text = generate_text(                                                                       │
│   34 │   prompt="What is the capital of United States?",                                         │
│   35 │   model_name="garage-bAInd/Platypus2-70B-instruct",                                       │
│   36 │   max_tokens=2048,                                                                        │
│                                                                                                  │
│ in generate_text:11                                                                              │
│                                                                                                  │
│    8 │   **kwargs                                                                                │
│    9 ):                                                                                          │
│   10 │   # The model_name can be the model path or the hugging face model repo id.               │
│ ❱ 11 │   model = AirLLMLlama2(model_name)                                                        │
│   12 │   input_tokens = model.tokenizer(                                                         │
│   13 │   │   [prompt],                                                                           │
│   14 │   │   return_tensors="pt",                                                                │
│                                                                                                  │
│ /home/josh/.local/lib/python3.10/site-packages/airllm/airllm.py:184 in __init__                  │
│                                                                                                  │
│   181 │   │   self.tokenizer = AutoTokenizer.from_pretrained(self.model_local_path)              │
│   182 │   │   self.tokenizer.pad_token = self.tokenizer.eos_token                                │
│   183 │   │   self.tokenizer.padding_side = "right"                                              │
│ ❱ 184 │   │   self.init_model()                                                                  │
│   185 │   │   self.layer_names = ["model.embed_tokens"] + [f"model.layers.{i}" for i in          │
│   186 │   │   │   │   │   │   │   │   │   │   │   │   │    range(len(self.model.model.layers))   │
│   187 │   │   self.max_seq_len = max_seq_len                                                     │
│                                                                                                  │
│ /home/josh/.local/lib/python3.10/site-packages/airllm/airllm.py:205 in init_model                │
│                                                                                                  │
│   202 │   │                                                                                      │
│   203 │   │   # Move buffers to device (not that much GPU memory used)                           │
│   204 │   │   for buffer_name, buffer in self.model.named_buffers():                             │
│ ❱ 205 │   │   │   set_module_tensor_to_device(self.model, buffer_name, self.running_device, va   │
│   206 │   │   │   │   │   │   │   │   │   │   dtype=self.running_dtype)                          │
│   207 │                                                                                          │
│   208 │   def load_layer_to_cpu(self, layer_name):         